ФОрмулы и вводная часть

In [43]:
import os
import scipy.io

In [84]:
import h5py

class DigitStructFile:
    """
    DigitStructFile - это просто оболочка для данных h5py. Это в основном ссылки
        inf: входной файл h5 matlab
        digitStructName Ссылка h5 на все имена файлов
        digitStructBbox Ссылка h5 на все структурированные данные
    """

    def __init__(self, inf):
        self.inf = h5py.File(inf, 'r')
        self.digitStructName = self.inf['digitStruct']['name']
        self.digitStructBbox = self.inf['digitStruct']['bbox']
        
    def getName(self, n):
        # getName returns the 'name' string for the n(th) digitStruct. 
        return self.inf[self.digitStructName[n][0]]
#TODO
# bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox. 
    def bboxHelper(self,attr):
        if (len(attr) > 1):
            attr = [self.inf[attr.value[j].item()].value[0][0] for j in range(len(attr))]
        else:
            attr = [attr.value[0][0]]
        return attr

# getBbox returns a dict of data for the n(th) bbox. 
    def getBbox(self,n):
        bbox = {}
        bb = self.digitStructBbox[n].item()
        bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
        bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
        bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
        bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
        bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
        return bbox

    def getDigitStructure(self,n):
        s = self.getBbox(n)
        s['name']=self.getName(n)
        return s

# getAllDigitStructure returns all the digitStruct from the input file.     
    def getAllDigitStructure(self):
        return [self.getDigitStructure(i) for i in range(len(self.digitStructName))]

# Return a restructured version of the dataset (one structure by boxed digit).
#
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#          'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#          'left', 'top' : position of bounding box
#          'width', 'height' : dimension of bounding box
#
# Note: We may turn this to a generator, if memory issues arise.
    def getAllDigitStructure_ByDigit(self):
        pictDat = self.getAllDigitStructure()
        result = []
        structCnt = 1
        for i in range(len(pictDat)):
            item = { 'filename' : pictDat[i]["name"] }
            figures = []
            for j in range(len(pictDat[i]['height'])):
               figure = {}
               figure['height'] = pictDat[i]['height'][j]
               figure['label']  = pictDat[i]['label'][j]
               figure['left']   = pictDat[i]['left'][j]
               figure['top']    = pictDat[i]['top'][j]
               figure['width']  = pictDat[i]['width'][j]
               figures.append(figure)
            structCnt = structCnt + 1
            item['boxes'] = figures
            result.append(item)
        return result

In [45]:
train_folders = './data/train'
test_folders = './data/test'
fin = os.path.join(train_folders, 'digitStruct.mat')
dsf = DigitStructFile(fin)
train_data = dsf.getAllDigitStructure_ByDigit()

In [87]:
dsf.inf[dsf.digitStructName[0][0]]
# a = dsf.inf['digitStruct']['name']
# a[0][0]
# dsf.getName(0)
# dsf.inf[a[0][0]]
dsf.inf[dsf.inf['digitStruct']['name'][0][0]]
# dsf.digitStructBbox[1][0]
# inf = h5py.File('./data/train/digitStruct.mat', 'r')
# digitStructName = inf['digitStruct']['name']
# digitStructBbox = inf['digitStruct']['bbox']

<HDF5 dataset "b": shape (5, 1), type "<u2">